In [1]:
import numpy as np
import pandas as pd

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [2]:
df = pd.read_csv("data.tsv",delimiter="\t", header=None)
df.head()

,0,1,2,3,4,5
0,100.0,2.000000,477.750000,3.464102,3.464102,10
1,50.0,4.250000,975.496844,1.477098,1.596796,10001
2,100.0,3.214286,3747.113679,1.022928,0.984565,15577
3,100.0,3.153846,4906.374000,0.831080,0.858073,22042
4,100.0,4.000000,132.100000,3.464102,3.464102,22227


In [3]:
data = df.values
data.shape

(1701, 6)

In [4]:
data = data / data.max(axis=0)
data.shape

(1701, 6)

In [5]:
def min_dist(clusters):
    m_d = 10000000
    for c1 in clusters:
        for c2 in clusters:
            if not c2 == c1:
                for x1 in clusters[c1]:
                    for x2 in clusters[c2]:
                        d = np.linalg.norm(x1-x2)
                        if d < m_d:
                            m_d = d
                    
    return m_d

def max_dist(clusters):
    m_d = 0
    for c1 in clusters:
        for c2 in clusters:
            if not c2 == c1:
                for x1 in clusters[c1]:
                    for x2 in clusters[c2]:
                        d = np.linalg.norm(x1-x2)
                        if d > m_d:
                            m_d = d
                    
    return m_d

def dunn(clusters):
    return min_dist(clusters)/max_dist(clusters)

In [6]:
import math

def Siq(w,X,q=2):
    return math.pow(np.mean([np.linalg.norm(x-w,q) \
                             for x in X]),1/q)

def Dijt(wi,wj,t=2):
    return np.linalg.norm(wi-wj,t)

def Riqt(clusters,centroids,i,q=2,t=2):
    return max(\
           [(Siq(centroids[i],clusters[i],q) +\
            Siq(centroids[j],clusters[j],q))\
            /Dijt(centroids[i],centroids[j],t) for \
              j in clusters.keys()-[i]])

def davies_bouldin(clusters,centroids):
    rs = [\
            Riqt(clusters,centroids,i) for \
            i in clusters]
    
    return np.mean(rs)

In [7]:
def Bk(centroids,nis,m):
    n = m.shape[0]
    bk = np.zeros((n,n))
    for c,ni in zip(centroids,nis):
        z = (c-m).reshape((1,n))
        bk += ni* (z.T * z)
        
    return bk

def Wk(clusters, centroids):
    n = centroids.shape[1]
    wk = np.zeros((n,n))
    
    for i in clusters:
        for x in clusters[i]:
            z = (x-centroids[i]).reshape(1,n)
            wk += z.T * z
            
    return wk

def calinski_harabasz(clusters, centroids,m):
    k = len(centroids)
    nis = [clusters[i].shape[0] for i in range(k)]

    bk = Bk(centroids,nis,m)
    
    wk = Wk(clusters, centroids)
    
    n = sum(nis)
    
    return (np.trace(bk)/(k-1))/(np.trace(wk)/(n-k))

In [8]:
df = []
m = np.mean(data,axis=0)
models = {}
for k in range(2,11):
    model = KMeans(n_clusters=k)
    clusters = {i: [] for i in range(k)}
    
    y = model.fit_predict(data)
    models[k] = model
    
    for c,x in zip(y,data):
        clusters[c].append(x)
        
    for c in clusters:
        clusters[c] = np.array(clusters[c])
    
    df.append({
        "1-K":k,
        "2-Dunn": dunn(clusters),
        "3-DB": davies_bouldin(clusters,model.cluster_centers_),
        "4-CH":calinski_harabasz(clusters,model.cluster_centers_,m),
        "5-SL":silhouette_score(data,y)
    })
        
df = pd.DataFrame(df)
df

,1-K,2-Dunn,3-DB,4-CH,5-SL
0,2,0.370576,1.359383,1917.547799,0.541584
1,3,0.015598,1.065287,2150.362182,0.587314
2,4,0.015452,1.075249,4325.263200,0.658898
3,5,0.010790,1.363054,4720.415403,0.653328
4,6,0.010790,1.509995,6460.105474,0.652840
5,7,0.006532,1.743611,6656.942135,0.640673
6,8,0.006532,1.883052,6874.657130,0.624277
7,9,0.006532,1.805076,7001.546239,0.631121
8,10,0.009974,2.062657,6942.918473,0.614515


In [9]:
df.style.background_gradient(cmap="Blues")

,1-K,2-Dunn,3-DB,4-CH,5-SL
0,2,0.370576,1.35938,1917.55,0.541584
1,3,0.0155981,1.06529,2150.36,0.587314
2,4,0.015452,1.07525,4325.26,0.658898
3,5,0.0107898,1.36305,4720.42,0.653328
4,6,0.0107898,1.50999,6460.11,0.65284
5,7,0.0065324,1.74361,6656.94,0.640673
6,8,0.0065324,1.88305,6874.66,0.624277
7,9,0.0065324,1.80508,7001.55,0.631121
8,10,0.00997393,2.06266,6942.92,0.614515


In [10]:
k = 4
model = models[k]
print("{} clusters:".format(k))

y = model.predict(data)

for c in range(k):
    d = data[y==c]
    stats_df = pd.DataFrame(d)
    print("\nC{} - {}\n".format(c, model.cluster_centers_[c]))
    print(stats_df.describe())

4 clusters:

C0 - [ 0.98284026  0.06229374  0.0290211   0.34959959  0.38158544  0.02654103]

                0           1           2           3           4           5
count  428.000000  428.000000  428.000000  428.000000  428.000000  428.000000
mean     0.982840    0.062294    0.029021    0.349600    0.381585    0.026541
std      0.074329    0.067237    0.075057    0.130558    0.136900    0.027271
min      0.250000    0.000000    0.001338    0.060074    0.078269    0.001001
25%      1.000000    0.034404    0.003921    0.248633    0.279109    0.008391
50%      1.000000    0.044500    0.007545    0.356753    0.389249    0.013594
75%      1.000000    0.066055    0.022249    0.460566    0.501039    0.029302
max      1.000000    1.000000    1.000000    0.593526    0.714286    0.097637

C1 - [ 0.99391971  0.0584906   0.01258958  0.43727342  0.4554304   0.93945604]

               0           1           2           3           4           5
count  552.00000  552.000000  552.000000  552.0